In [1]:
nLookBackDays=5
nForecastDays=1
hoursPerDay = 24
nForecastHours = nForecastDays * hoursPerDay

# 참고
- https://keras.io/examples/timeseries/timeseries_classification_transformer/
- 위의 코드는 트랜스포머 모델을 제안한 논문에 나온 모델을 그대로 구현한 것이라고 함
- 위의 코드는 시계열 분류 모델을 구현한 것인데, 최종 출력층을 약간 변형하여 시계열을 예측하는 모델로 만들어서 사용함

In [2]:
# 추가 패키지
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
plt.style.use('ggplot')

# 1 데이터 불러오기

In [3]:
# 데이터 불러오기
import pandas as pd
#pdDataset = pd.read_excel('../1) 한전 전력 전처리/3B)HourlyPower(SeparateDate)(Day1Start)(NoWeekend).xlsx', 
#                          sheet_name = 'data')
pdDataset = pd.read_excel('../1) 한전 전력 전처리/3A)HourlyPower(SeparateDate)(AnyDayStart)(NoWeekend).xlsx', 
                          sheet_name = 'data')

In [4]:
datalen = pdDataset.shape[0]

In [ ]:
# 테스트 ; 불러온 데이터 확인
if False:
    print(pdDataset.shape)
    print(pdDataset.head())

In [ ]:
# 테스트
if False:
    print(pdDataset["TotalPower(kWh)"][0:5],end="\n\n")
    print(pdDataset["PeakPower(kW)"][0:5],end="\n\n")
    print(pdDataset["Date"][0:5],end="\n\n")
    print(pdDataset["Year"][0:5],end="\n\n")
    print(pdDataset["Month"][0:5],end="\n\n")
    print(pdDataset["Day"][0:5],end="\n\n")
    print(pdDataset["Yoil"][0:5])

In [ ]:
if False:
    plt.figure(figsize=(20,10))
    plt.plot(range(datalen), pdDataset["TotalPower(kWh)"].values)
    plt.ylabel('Power consumption')
    plt.title("Total power use (per hour)")
    plt.show()

    plt.figure(figsize=(20,10))
    plt.plot(range(datalen), pdDataset["PeakPower(kW)"].values, label="PEAK power")
    plt.ylabel('Peak power')
    #plt.legend()
    plt.title("Peak power (per hour)")
    plt.show()

### 데이터를 하루(24개) 단위로 묶어서 저장하자

In [5]:
# 필요한 상수 정의하기
numDaysTotal = int(datalen/hoursPerDay)
print("Number of days in total : ", numDaysTotal)

Number of days in total :  181


In [6]:
perDayTotalPowerList = [] # 하루 24개 데이터를 리스트 형태로 저장(총전력)
perDayTotalPower = [] # 하루 24시간 총 전력을 하나의 값으로 해서 저장
perDayPeakPowerList = [] # 하루 24개 데이터를 리스트 형태로 저장(peak전력)
perDayPeakPower = [] # 하루 중 최대 전력 하나를 골라서 저장
perDayDateInfo = [] # 년.월.일 정보를 저장

for nthDay in range(numDaysTotal):
    currHourTotalPowerList, currHourPeakPowerList = [], []
    
    currDate = str(pdDataset["Year"][nthDay*hoursPerDay]) + "." \
               + str(pdDataset["Month"][nthDay*hoursPerDay]) + "." \
               + str(pdDataset["Day"][nthDay*hoursPerDay])
    perDayDateInfo.append(currDate)
    
    for hour in range(hoursPerDay):
        index = nthDay*hoursPerDay + hour
        currHourTotalPowerList.append(pdDataset["TotalPower(kWh)"][index])
        currHourPeakPowerList.append(pdDataset["PeakPower(kW)"][index])
   
    perDayTotalPowerList.append(currHourTotalPowerList)
    perDayTotalPower.append(sum(currHourTotalPowerList))
    perDayPeakPowerList.append(currHourPeakPowerList)
    perDayPeakPower.append(max(currHourPeakPowerList))
    
perDayTotalPowerList = np.array(perDayTotalPowerList)
perDayTotalPower = np.array(perDayTotalPower)
perDayPeakPowerList = np.array(perDayPeakPowerList)
perDayPeakPower = np.array(perDayPeakPower)
perDayDateInfo = np.array(perDayDateInfo)

In [ ]:
if False: 
    print(perDayTotalPowerList)

In [ ]:
if False: 
    print(perDayTotalPower)

In [ ]:
if False: 
    print(perDayPeakPowerList)

In [ ]:
if False: 
    print(perDayPeakPower)

In [ ]:
if False: 
    print(perDayDateInfo)

# 2 전처리

## 2.1 outlier 제거, 2.2 누락된 값 복원하기, ...
- 별도의 전처리가 필요 없는 것으로 생각되어, 전처리는 하지 않음
- 데이터의 수가 충분히 많으므로, 소수의 outlier의 영향이 최소화 됨

## 2.2 전처리된 데이터 시각화

# 3. Transformer로 예측하기(Univariate)

## 필요 패키지 import

In [7]:
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error

## 학습에 사용하기 위해 차원을 고려해서 데이터 준비하기

### 전체 데이터를 학습용, 테스트용으로 분리하기

In [8]:
"""
학습에 사용할 샘플 수 정하기
터미널 출력 결과를 보고 비중(portion)을 적절히 골랐음
"""
portion = 0.945
howManyDaysToTrain = int(numDaysTotal * portion)

if True:
    print("Train for " + str(howManyDaysToTrain) 
          + " days out of " + str(numDaysTotal) + " days")
    print("Forecast for %d days out of %d days." % (numDaysTotal
                                                    - howManyDaysToTrain, numDaysTotal))

Train for 171 days out of 181 days
Forecast for 10 days out of 181 days.


In [9]:
from MyUtils import split_dataset

### 총 전력량 데이터를 split : 시간 단위

In [ ]:
if False:
    rawHourlyTotalPower = pdDataset["TotalPower(kWh)"].values
    hourlyTotalPowerTrain, hourlyTotalPowerTest \
    = split_dataset(rawHourlyTotalPower, howManyDaysToTrain, hoursPerDay)

In [ ]:
if False: 
    print(hourlyTotalPowerTrain)

In [ ]:
if False: 
    print(hourlyTotalPowerTest)

### 전력 PEAK 데이터를 split : 시간 단위

In [10]:
rawHourlyPeakPower = pdDataset["PeakPower(kW)"].values
hourlyPeakPowerTrain, hourlyPeakPowerTest \
= split_dataset(rawHourlyPeakPower, howManyDaysToTrain, hoursPerDay)

In [ ]:
print(str(int(howManyDaysToTrain*0.9)) + " out of " + str(howManyDaysToTrain))
print(rawHourlyPeakPower.shape)
print(hourlyPeakPowerTrain.shape)

In [ ]:
if False: 
    print(hourlyPeakPowerTrain)

In [ ]:
if False: 
    print(hourlyPeakPowerTest)

### 총 전력량 데이터를 split ; 일 단위

In [ ]:
# 필요 없음
# 어차피, 모든 연속적인 24시간을 단위로 할거라서, 일 단위(1-24시)로 계산한 데이터는 큰 필요가 없고, 
# 그때그때 직접 계산하여서 사용하기

### 전력 PEAK 데이터를 split : 일 단위

In [ ]:
# 필요 없음
# 어차피, 모든 연속적인 24시간을 단위로 할거라서, 일 단위(1-24시)로 계산한 데이터는 큰 필요가 없고, 
# 그때그때 직접 계산하여서 사용하기

### 년.월.일 데이터를 split
- 이 데이터는 학습에 사용하지는 않음
- 추가적인 검증 등의 목적으로 사용할건데, 여튼 다른 데이터와 마찬가지로 split을 해 둬야
  다른 데이터와 같은 인덱스를 사용할 수 있으니까 일단은 split을 해 두는 것임

In [ ]:
# 필요 없음
# 어차피, 모든 연속적인 24시간을 단위로 할거라서, 일 단위(1-24시)로 계산한 데이터는 큰 필요가 없고, 
# 그때그때 직접 계산하여서 사용하기

### 학습, 테스트 데이터를 슬라이딩 윈도우 구조에 맞게 정리 (+ 그리고, 과거 몇일간의 데이터를 입력으로 줄 지도 결정)

In [ ]:
# 테스트
if False: 
    d = hourlyTotalPowerTrain
    print(d[0])
    print(d[1])
    print(np.concatenate((d[0],d[1],d[2],d[3]), axis=0))
    print(np.concatenate((d[0:4]), axis=0))

In [11]:
from MyUtils import to_supervisedDaily # 1-24시간 단위로 분할
from MyUtils import to_supervisedContinuousHours # 연속된 모든 24시간 단위로 분할

### 학습에 사용할 수 있는 형태로 데이터 준비하기

### (시간 당) 총 전력 사용량

In [ ]:
if False:
    XHourlyTotalPowerTrain, yHourlyTotalPowerTrain \
    = to_supervisedContinuousHours(hourlyTotalPowerTrain, nLookBackDays, nForecastDays)

    # 검증은 1-24시간 단위의 데이터로 할 것임 >> to_supervisedDaily 함수를 사용
    XHourlyTotalPowerTest, yHourlyTotalPowerTest \
    = to_supervisedDaily(hourlyTotalPowerTest, nLookBackDays, nForecastDays)

In [ ]:
if False:
    # 샘플 수가 얼마지?
    print(XHourlyTotalPowerTrain.shape)

#### 조건부 입력(전력  사용량 총량)을 위해 추가로 데이터 셋 마련하기
- 1-24시 단위가 아니다 (y 데이터셋에 들어있는 값의 총합을 직접 구하고, 이것으로 데이터 셋을 마련하자)
- 예측할 24시간에 대한 전력 사용량 총합을 미리 계산하고, 이를 조건부 입력으로 사용

In [ ]:
if False:
    # 예측할 24시간에 대한 전력 사용량 총합을 미리 계산하고, 이를 조건부 입력으로 사용
    XContinuousDailyTotalPowerTrain = []
    #print(yHourlyTotalPowerTrain.shape) # (3696, 24, 1)

    numSamples = yHourlyTotalPowerTrain.shape[0]
    for i in range(numSamples):
        sample = yHourlyTotalPowerTrain[i]
        XContinuousDailyTotalPowerTrain.append(sum(sample))
        if False:
            print(sample)
            print(sum(sample))
            break

    XContinuousDailyTotalPowerTrain = np.array(XContinuousDailyTotalPowerTrain)
    #print(XContinuousDailyTotalPowerTrain.shape)

In [ ]:
if False:
    XContinuousDailyTotalPowerTest = []
    #print(yHourlyTotalPowerTest.shape) # (5, 24, 1)

    numSamples = yHourlyTotalPowerTest.shape[0]
    for i in range(numSamples):
        sample = yHourlyTotalPowerTest[i]
        XContinuousDailyTotalPowerTest.append(sum(sample))

    XContinuousDailyTotalPowerTest = np.array(XContinuousDailyTotalPowerTest)
    #print(XContinuousDailyTotalPowerTest.shape)

### (시간 당) PEAK 전력

In [12]:
XHourlyPeakPowerTrain, yHourlyPeakPowerTrain \
= to_supervisedContinuousHours(hourlyPeakPowerTrain, nLookBackDays, nForecastDays)

# 검증은 1-24시간 단위의 데이터로 할 것임 >> to_supervisedDaily 함수를 사용
XHourlyPeakPowerTest, yHourlyPeakPowerTest \
= to_supervisedDaily(hourlyPeakPowerTest, nLookBackDays, nForecastDays)

In [ ]:
# 샘플 수가 얼마지?
print(XHourlyPeakPowerTrain.shape)
print(yHourlyPeakPowerTrain.shape)
print(XHourlyPeakPowerTest.shape)
print(yHourlyPeakPowerTest.shape)

#### 조건부 입력(전력  사용량 총량)을 위해 추가로 데이터 셋 마련하기
- 1-24시 단위가 아니다 (y 데이터셋에 들어있는 값의 max를 직접 구하고, 이것으로 데이터 셋을 마련하자)
- 예측할 24시간에 대한 전력 사용량 max를 미리 계산하고, 이를 조건부 입력으로 사용

In [13]:
# 예측할 24시간에 대한 전력 peak를 미리 계산하고, 이를 조건부 입력으로 사용
XContinuousDailyPeakPowerTrain = []
#print(XContinuousDailyPeakPowerTrain.shape)

numSamples = yHourlyPeakPowerTrain.shape[0]
for i in range(numSamples):
    sample = yHourlyPeakPowerTrain[i]
    XContinuousDailyPeakPowerTrain.append(max(sample))  # 최대값을 리턴
    if i == -1:
        print(sample)
        print(max(sample))
        #break

XContinuousDailyPeakPowerTrain = np.array(XContinuousDailyPeakPowerTrain)
#print(XContinuousDailyPeakPowerTrain.shape)

In [14]:
# 예측할 24시간에 대한 전력 peak를 미리 계산하고, 이를 조건부 입력으로 사용
XContinuousDailyPeakPowerTest = []
#print(XContinuousDailyPeakPowerTest.shape)

numSamples = yHourlyPeakPowerTest.shape[0]
for i in range(numSamples):
    sample = yHourlyPeakPowerTest[i]
    XContinuousDailyPeakPowerTest.append(max(sample))  # 최대값을 리턴
    if i == -1:
        print(sample)
        print(max(sample))
        #break

XContinuousDailyPeakPowerTest = np.array(XContinuousDailyPeakPowerTest)
#print(XContinuousDailyPeakPowerTest.shape)

### (일 단위) 총 전력 사용량

In [ ]:
# 연속적인 24시간 단위의 데이터 셋 에서는 필요 없음

### (일 단위) PEAK 전력

In [ ]:
# 연속적인 24시간 단위의 데이터 셋 에서는 필요 없음

### (일 단위) 날짜 정보

In [ ]:
# 연속적인 24시간 단위의 데이터 셋 에서는 필요 없음

## Transformer 모델 만들기

In [15]:
from tensorflow import keras
from MyModels import *
#from ForcingMaxUtils import build_transformer_conditionalInput_maxOutput_model_gAvgPooling

In [16]:
def build_transformer_conditionalInput_maxOutput_model_gAvgPooling(
    # 전력 PEAK를 조건부 입력으로 제공 + 출력층에서 시계열 데이터의 PEAK를 출력
    input_shape, conditional_input_shape, num_outputs,
    head_size, num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0, mlp_dropout=0,
    ):
    
    LayerNormEps = 1e-6
    """
    인코더 
    """
    inputs = keras.Input(shape=input_shape, name="ts_input") # 시계열 학습 데이터
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    # PEAK는 최대값을 예측하는 문제니까, max pooling이 더 좋으려나?
    #x = layers.GlobalMaxPooling1D(data_format="channels_first")(x)
    #x = layers.MaxPooling1D(pool_size=4, strides=1, padding='same', data_format="channels_first")(x)
    
    # 조건부 입력(일간 총 사용량 예측치) 데이터를 추가로 입력 받음
    conditional_inputs = keras.Input(shape=conditional_input_shape, name="ts_max_input")
    # 조건부 입력(일간 총 사용량)의 단위가 크고, 다른 특징들은 이미 정규화가 되어있어서
    # 조건부 입력을 그대로 사용하면 이로 인한 업데이트가 너무나 과도해 질 수 있음
    # 따라서, 조건부 입력에 대해서도 정규화를 실시함
    normalized_conditional_inputs \
    = layers.LayerNormalization(epsilon=LayerNormEps)(conditional_inputs)
    # 인코더의 최종 출력 + 조건부 입력을 인코더 계층의 최종 출력으로 하고, 디코더로 전달
    x = x + normalized_conditional_inputs
    
    """
    디코더
    """
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu",
                         kernel_initializer='random_normal',
                         bias_initializer='zeros')(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    # 양수만 나와야 하니까, linear 대신 relu
    timeseries_outputs = layers.Dense(num_outputs, activation="relu",
                                      kernel_initializer='random_normal',
                                      bias_initializer='zeros',
                                      name='ts_output')(x) 
    
    #시계열 출력을 다시 입력으로 받아서, 출력의 element-wise max을 계산
    max_outputs = layers.Lambda(lambda v: K.max(v), 
                                output_shape=(1,1),
                                name="ts_max")(timeseries_outputs)
    # 최종 모델을 리턴
    return keras.Model(inputs = [inputs, conditional_inputs], 
                       outputs = [timeseries_outputs, max_outputs])

In [17]:
from MyMetrics import mape, smape, rmse, mae, coeff_determination
import os
from MyUtils import plotTrainingProgress
from ForcingMaxUtils import test_predict_mergedInput_mergedOutput_forcingMax

## PEAK 전력 예측하기

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1" # first gpu for Case 1 code
#os.environ["CUDA_VISIBLE_DEVICES"]="0" # first gpu for Case 1 code
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # second gpu for Case 2 code

In [20]:
dropoutRate = 0.3

### PEAK : Case 2 (Trial 1)

In [ ]:
model_peakpower_case2_1 = build_transformer_conditionalInput_maxOutput_model_gAvgPooling(
    input_shape = XHourlyPeakPowerTrain.shape[1:],
    conditional_input_shape = XContinuousDailyPeakPowerTrain.shape[1:],
    num_outputs = nForecastHours,
    head_size=256,
    num_heads=4,
    ff_dim=4, # 8 : 성능이 오히려 떨어짐
    num_transformer_blocks=6,
    #mlp_units=[128, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_units=[512, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_dropout=dropoutRate, # 0.4
    dropout=dropoutRate) # 0.25
"""
시계열 max가  예측 peak와 같아지는 것을 강제하는 가중치
peak 예측도 중요하고, 시계열 예측도 중요(몇시에 발생하는지)하므로
peak 예측에 대한 가중치를 높여나가면서 실험 함
"""
ts_max_weight = 0.1
lossWeight = {"ts_output":1.0-ts_max_weight,"ts_max":ts_max_weight}

# ts : time-series
# ts_max의 영향을 줄이기 위해서 가중치 작게 설정했고, mae를 사용함
# => mse로 변경
# Adam + decay를 사용할거라면, 초기 lr은 큰 값으로 설정해도 되겠다.
model_peakpower_case2_1.compile(loss = {"ts_output" : 'mse', "ts_max" : 'mae'},
                              optimizer = keras.optimizers.Adam(learning_rate = 0.001),
                              metrics = {"ts_output":mae,"ts_max":mae},
                              loss_weights = lossWeight)

callbacks = [keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True)]

training_history_peakpower_case2_1 = \
model_peakpower_case2_1.fit(
    x = {"ts_input" : XHourlyPeakPowerTrain, 
         "ts_max_input" : XContinuousDailyPeakPowerTrain},
    y = {"ts_output" : yHourlyPeakPowerTrain, 
         "ts_max" : XContinuousDailyPeakPowerTrain},    
    validation_data=(
        {"ts_input" : XHourlyPeakPowerTest, 
         "ts_max_input" : XContinuousDailyPeakPowerTest},
        {"ts_output" : yHourlyPeakPowerTest, 
         "ts_max" : XContinuousDailyPeakPowerTest},        
    ),
    epochs = 100,  # 어차피 early stopping이 적용될거라 epoch 커도 ok
    batch_size = 32,  # 64보다, 128일때 속도가 더 빠르다...? 그런데, 32가 가장 일반적으로 사용하는 숫자라고 함
    callbacks = callbacks,
    verbose = 2,
    shuffle = True) # True로 하더라도, 검증 데이터는 섞이지 않음 (True가 결과가 더 좋음)

plotTrainingProgress(training_history=training_history_peakpower_case2_1, 
                     title="Case 2 ; peak power (%d LookBack)"%(nLookBackDays))

# 예측하기 (총전력)
# 평가는 1-24시 단위로 자른 데이터로...
test_predict_mergedInput_mergedOutput_forcingMax(model_peakpower_case2_1, 
             [XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             "Case 2 : peak power (%d LookBack)"%(nLookBackDays))

# 평가는 1-24시 단위로 자른 데이터로...
model_peakpower_case2_1.evaluate([XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               verbose=1)

### PEAK : Case 2 (Trial 2)

In [ ]:
model_peakpower_case2_2 = build_transformer_conditionalInput_maxOutput_model_gAvgPooling(
    input_shape = XHourlyPeakPowerTrain.shape[1:],
    conditional_input_shape = XContinuousDailyPeakPowerTrain.shape[1:],
    num_outputs = nForecastHours,
    head_size=256,
    num_heads=4,
    ff_dim=4, # 8 : 성능이 오히려 떨어짐
    num_transformer_blocks=6,
    #mlp_units=[128, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_units=[512, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_dropout=dropoutRate, # 0.4
    dropout=dropoutRate) # 0.25
"""
시계열 max가  예측 peak와 같아지는 것을 강제하는 가중치
peak 예측도 중요하고, 시계열 예측도 중요(몇시에 발생하는지)하므로
peak 예측에 대한 가중치를 높여나가면서 실험 함
"""
ts_max_weight = 0.1
lossWeight = {"ts_output":1.0-ts_max_weight,"ts_max":ts_max_weight}

# ts : time-series
# ts_max의 영향을 줄이기 위해서 가중치 작게 설정했고, mae를 사용함
# => mse로 변경
# Adam + decay를 사용할거라면, 초기 lr은 큰 값으로 설정해도 되겠다.
model_peakpower_case2_2.compile(loss = {"ts_output" : 'mse', "ts_max" : 'mae'},
                              optimizer = keras.optimizers.Adam(learning_rate = 0.001),
                              metrics = {"ts_output":mae,"ts_max":mae},
                              loss_weights = lossWeight)

callbacks = [keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True)]

training_history_peakpower_case2_2 = \
model_peakpower_case2_2.fit(
    x = {"ts_input" : XHourlyPeakPowerTrain, 
         "ts_max_input" : XContinuousDailyPeakPowerTrain},
    y = {"ts_output" : yHourlyPeakPowerTrain, 
         "ts_max" : XContinuousDailyPeakPowerTrain},    
    validation_data=(
        {"ts_input" : XHourlyPeakPowerTest, 
         "ts_max_input" : XContinuousDailyPeakPowerTest},
        {"ts_output" : yHourlyPeakPowerTest, 
         "ts_max" : XContinuousDailyPeakPowerTest},        
    ),
    epochs = 100,  # 어차피 early stopping이 적용될거라 epoch 커도 ok
    batch_size = 32,  # 64보다, 128일때 속도가 더 빠르다...? 그런데, 32가 가장 일반적으로 사용하는 숫자라고 함
    callbacks = callbacks,
    verbose = 2,
    shuffle = True) # True로 하더라도, 검증 데이터는 섞이지 않음 (True가 결과가 더 좋음)

plotTrainingProgress(training_history=training_history_peakpower_case2_2, 
                     title="Case 2 ; peak power (%d LookBack)"%(nLookBackDays))

# 예측하기 (총전력)
# 평가는 1-24시 단위로 자른 데이터로...
test_predict_mergedInput_mergedOutput_forcingMax(model_peakpower_case2_2, 
             [XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             "Case 2 : peak power (%d LookBack)"%(nLookBackDays))

# 평가는 1-24시 단위로 자른 데이터로...
model_peakpower_case2_2.evaluate([XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               verbose=1)

### PEAK : Case 2 (Trial 3)

In [ ]:
model_peakpower_case2_3 = build_transformer_conditionalInput_maxOutput_model_gAvgPooling(
    input_shape = XHourlyPeakPowerTrain.shape[1:],
    conditional_input_shape = XContinuousDailyPeakPowerTrain.shape[1:],
    num_outputs = nForecastHours,
    head_size=256,
    num_heads=4,
    ff_dim=4, # 8 : 성능이 오히려 떨어짐
    num_transformer_blocks=6,
    #mlp_units=[128, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_units=[512, 256, 64], # 문제의 복잡도 증가로 인해, 디코더 쪽에서 MLP 계층을 늘렸음
    mlp_dropout=dropoutRate, # 0.4
    dropout=dropoutRate) # 0.25
"""
시계열 max가  예측 peak와 같아지는 것을 강제하는 가중치
peak 예측도 중요하고, 시계열 예측도 중요(몇시에 발생하는지)하므로
peak 예측에 대한 가중치를 높여나가면서 실험 함
"""
ts_max_weight = 0.1
lossWeight = {"ts_output":1.0-ts_max_weight,"ts_max":ts_max_weight}

# ts : time-series
# ts_max의 영향을 줄이기 위해서 가중치 작게 설정했고, mae를 사용함
# => mse로 변경
# Adam + decay를 사용할거라면, 초기 lr은 큰 값으로 설정해도 되겠다.
model_peakpower_case2_3.compile(loss = {"ts_output" : 'mse', "ts_max" : 'mae'},
                              optimizer = keras.optimizers.Adam(learning_rate = 0.001),
                              metrics = {"ts_output":mae,"ts_max":mae},
                              loss_weights = lossWeight)

callbacks = [keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True)]

training_history_peakpower_case2_3 = \
model_peakpower_case2_3.fit(
    x = {"ts_input" : XHourlyPeakPowerTrain, 
         "ts_max_input" : XContinuousDailyPeakPowerTrain},
    y = {"ts_output" : yHourlyPeakPowerTrain, 
         "ts_max" : XContinuousDailyPeakPowerTrain},    
    validation_data=(
        {"ts_input" : XHourlyPeakPowerTest, 
         "ts_max_input" : XContinuousDailyPeakPowerTest},
        {"ts_output" : yHourlyPeakPowerTest, 
         "ts_max" : XContinuousDailyPeakPowerTest},        
    ),
    epochs = 100,  # 어차피 early stopping이 적용될거라 epoch 커도 ok
    batch_size = 32,  # 64보다, 128일때 속도가 더 빠르다...? 그런데, 32가 가장 일반적으로 사용하는 숫자라고 함
    callbacks = callbacks,
    verbose = 2,
    shuffle = True) # True로 하더라도, 검증 데이터는 섞이지 않음 (True가 결과가 더 좋음)

plotTrainingProgress(training_history=training_history_peakpower_case2_3, 
                     title="Case 2 ; peak power (%d LookBack)"%(nLookBackDays))

# 예측하기 (총전력)
# 평가는 1-24시 단위로 자른 데이터로...
test_predict_mergedInput_mergedOutput_forcingMax(model_peakpower_case2_3, 
             [XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             "Case 2 : peak power (%d LookBack)"%(nLookBackDays))

# 평가는 1-24시 단위로 자른 데이터로...
model_peakpower_case2_3.evaluate([XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
                               verbose=1)

In [22]:
def test_predict_mergedInput_mergedOutput_forcingMax_multiplemodels(model1, model2, model3, Xmerged, ymerged, title, _ylim=None):
    # X 데이터 분리
    X = Xmerged[0]
    conditionalX = Xmerged[1]
    # y 데이터 분리
    y = ymerged[0]
    maxy = ymerged[1]

    ts_pred = [] # 시계열 예측값을 저장할 리스트
    actual = y.reshape(y.shape[0] * y.shape[1]) # 시계열 정답을 저장할 리스트
    
    ts_max_pred = [] # 24시간 예측 단위로, 예측으로 생성한 시계열 데이터의 max 저장할 리스트
    max_y_values = [] # 24시간 예측 단위로, 정답에 해당하는 시계열 데이터의 max 저장할 리스트
    
    for i in range(X.shape[0]):
        x_sample1 = X[i].reshape(1, len(X[i]), 1)
        x_sample2 = conditionalX[i].reshape(1, len(conditionalX[i]), 1)
        
        [y_hat1, y_hat_max1] = model1.predict([x_sample1, x_sample2])
        [y_hat2, y_hat_max2] = model2.predict([x_sample1, x_sample2])
        [y_hat3, y_hat_max3] = model3.predict([x_sample1, x_sample2])
            
        y_hat_values1 = y_hat1[0].reshape(len(y_hat1[0]),).tolist()
        y_hat_values2 = y_hat2[0].reshape(len(y_hat2[0]),).tolist()
        y_hat_values3 = y_hat3[0].reshape(len(y_hat3[0]),).tolist()
        
        y_hat_values = []
        for j in range(len(y_hat_values1)):
            y_hat_values.append(max(y_hat_values1[j],y_hat_values2[j],y_hat_values3[j]))

        if len(ts_pred) == 0:
            ts_pred = y_hat_values
        else:
            ts_pred = ts_pred + y_hat_values
            
        ts_max_pred.append(max(y_hat_max1,y_hat_max2,y_hat_max3))
        max_y_values.append(maxy[i][0])
            
    # 시계열 데이터 예측값을 plot
    print("TimeSeries MAE : %d"%(int(mae(actual, ts_pred))))
    #plt.figure(figsize=(20,10))
    plt.figure()
    plt.plot(actual, label='actual')
    plt.plot(ts_pred, label='forecast')
    plt.title(title) 
    plt.legend()
    plt.xlabel('hours')
    if _ylim is not None:
        plt.ylim(_ylim)
    plt.show()
        
    # 24시간 예측 단위로, 시계열 데이터의 총합을 plot
    print("TimeSeriesSum MAE : %d"%(int(mae(np.array(max_y_values), np.array(ts_max_pred)))))
    #print(sum_y_values)
    #print(ts_sum_pred)
    plt.figure()
    plt.bar(np.arange(len(max_y_values))-0.1, max_y_values, width=0.3, label='actual')
    plt.bar(np.arange(len(ts_max_pred))+0.1, ts_max_pred, width=0.3, label='forecast')
    plt.legend()
    plt.title("Comparison: daily power max")
    plt.xlabel('days')
    plt.show()

In [ ]:
# 예측하기 (총전력)
# 평가는 1-24시 단위로 자른 데이터로...
test_predict_mergedInput_mergedOutput_forcingMax_multiplemodels(model_peakpower_case2_1, model_peakpower_case2_2, model_peakpower_case2_3, 
             [XHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             [yHourlyPeakPowerTest,XContinuousDailyPeakPowerTest], 
             "Case 2 : peak power (%d LookBack)"%(nLookBackDays))